In [1]:
#Q8
ans = 1
i=1126
while ans >= 0.5:
    i -= 1
    ans = ans*(i/1126)
print(i)


1087


In [ ]:
implement the AdaBoost-Stump algorithm
Use a total of T = 500 iterations (not stop earlier than 500), calculate Ein and Eout with 0/1 error

In [ ]:
(1) For any feature i, sort all the Xn,i values to x[n],i such that x[n],i ≤ x[n+1],i.
(2) Consider thresholds within −∞ and all the midpoints x[n],i + x[n+1],i / 2. 
Test those thresholds with s ∈ {−1, +1} to determine the best (s, θ)combination minimizes Euin using feature i.
(3) Pick the best (s, i, θ) combination by enumerating over all possible i.

In [ ]:
AdaBoost 使用的 Weak Learner 是 Decision Stump，是一個只能在平面上切一條水平線或垂直線的 Weak Learner，
詳細演算法如下，每個假設就是要學習出是一條水平線或垂直線（direction s），切在哪個 feature（feature i），
切在那個值（threshold theta）。

In [1]:
#Q11-Q16
x_train = []
y_train = []
x_test = []
y_test = []

f = open('hw6_train.dat.txt')
for line in f.readlines():
    x_train.append([float(i) for i in line.split(' ')[:10]])
    y_train.append(float(line.split(' ')[-1][:3]))
f.close

f = open('hw6_test.dat.txt')
for line in f.readlines():
    x_test.append([float(i) for i in line.split(' ')[:10]])
    y_test.append(float(line.split(' ')[-1][:3]))
f.close

import numpy as np
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [16]:
#Q11
N = x_train.shape[0]
dim = x_train.shape[1]
u = np.array([(1.0 / N)] * N)
Ein = 1

for i in range(dim):
    Ein_i = 1
    threshold_i = 0
    feature_i = x_train[:,i]
    data_i = np.transpose(np.array([feature_i, y_train]))
    sorted_data_i = data_i[np.argsort(data_i[:, 0])]
    
    thresholds = np.array([float("-inf")] + [(sorted_data_i[j,0] + sorted_data_i[j+1,0]) / 2 for j in range(N-1)])
    
    for threshold in thresholds:
        y_pos = np.where(sorted_data_i[:,0] > threshold, 1, -1)
        y_neg = np.where(sorted_data_i[:,0] < threshold, 1, -1)
        Epos = sum((y_pos != sorted_data_i[:,1]) * u)
        Eneg = sum((y_neg != sorted_data_i[:,1]) * u)
        if Epos > Eneg and Ein_i > Eneg:
            Ein_i = Eneg
        elif Epos < Eneg and Ein_i > Epos:
            Ein_i = Epos
    
    if Ein > Ein_i:
        Ein = Ein_i

print(Ein)
#choose c

0.3740000000000003


In [2]:
#Q12 13 14 15 16
import math 
def update_u(u,s,thres,feature_idx,y):
    if s == -1:
        y_pred = np.where(x_train[:,feature_idx] < thres, 1, -1)
    else:
        y_pred = np.where(x_train[:,feature_idx] > thres, 1, -1)
    incorrect = np.where(y_pred != y, 1, 0)
        
    e_t = sum(incorrect * u) / sum(u)
    diamond_t = math.sqrt((1-e_t)/e_t)
    
    new_u = np.where(y_pred == y, 1/diamond_t, diamond_t) * u
    
    alpha = np.log(diamond_t)
    
    return new_u,alpha


In [4]:
#Q12
N = x_train.shape[0]
dim = x_train.shape[1]
u = np.array([(1.0 / N)] * N)
s = 1
thres = 0
max_Ein = 0
feature_idx = 0

for t in range(500):
    Ein = 1

    for i in range(dim):
        Ein_i = 1
        threshold_i = 0
        data_i = np.transpose(np.array([x_train[:,i], y_train]))
        sorted_data_i = data_i[np.argsort(data_i[:, 0])]
        sorted_u_i = u[np.argsort(data_i[:, 0])]

        thresholds = np.array([float("-inf")] + [(sorted_data_i[j,0] + sorted_data_i[j+1,0]) / 2 for j in range(N-1)])
        
        for threshold in thresholds:
            y_pos = np.where(sorted_data_i[:,0] > threshold, 1, -1)
            y_neg = np.where(sorted_data_i[:,0] < threshold, 1, -1)
            Epos = sum((y_pos != sorted_data_i[:,1]) * sorted_u_i)
            Eneg = sum((y_neg != sorted_data_i[:,1]) * sorted_u_i)
            if Epos > Eneg:
                if Ein_i > Eneg:
                    Ein_i = Eneg
                    s_i = -1
                    threshold_i = threshold
            else:
                if Ein_i > Epos:
                    Ein_i = Epos
                    s_i = 1
                    threshold_i = threshold

        if Ein > Ein_i:
            Ein = Ein_i
            s = s_i
            thres = threshold_i
            feature_idx = i
            
    u, alpha = update_u(u,s,thres,feature_idx, y_train)
            
    if s == 1:
        pred_y = np.where(x_train[:,feature_idx] > thres, 1, -1)
    else:
        pred_y = np.where(x_train[:,feature_idx] < thres, 1, -1)
    
    Ein_01 = np.sum(pred_y != y_train)/N
    if Ein_01 > max_Ein:
        max_Ein = Ein_01
        
print(max_Ein)
#choose e

0.591


In [6]:
#Q13
def Ein_Gt(classifiers):
    T = len(classifiers)
    x_train_label = np.array([0.0] * x_train.shape[0])
    for classifier in classifiers:
        if classifier[1] == 1:
            pred_y = np.where(x_train[:,classifier[3]] > classifier[2], 1, -1)
        else:
            pred_y = np.where(x_train[:,classifier[3]] < classifier[2], 1, -1)
        
        x_train_label += classifier[0]*pred_y
    x_train_label = np.where(x_train_label > 0, 1, -1)
    Ein_G = sum(x_train_label != y_train)/ x_train.shape[0]
    return Ein_G

In [6]:
#Q13
times = [60, 160, 260, 360, 460]
classifiers = []
N = x_train.shape[0]
dim = x_train.shape[1]
u = np.array([(1.0 / N)] * N)
s = 1
thres = 0
max_Ein = 0
feature_idx = 0

for t in range(460):
    Ein = 1

    for i in range(dim):
        Ein_i = 1
        threshold_i = 0
        data_i = np.transpose(np.array([x_train[:,i], y_train]))
        sorted_data_i = data_i[np.argsort(data_i[:, 0])]
        sorted_u_i = u[np.argsort(data_i[:, 0])]

        thresholds = np.array([float("-inf")] + [(sorted_data_i[j,0] + sorted_data_i[j+1,0]) / 2 for j in range(N-1)])

        for threshold in thresholds:
            y_pos = np.where(sorted_data_i[:,0] > threshold, 1, -1)
            y_neg = np.where(sorted_data_i[:,0] < threshold, 1, -1)
            Epos = sum((y_pos != sorted_data_i[:,1]) * sorted_u_i)
            Eneg = sum((y_neg != sorted_data_i[:,1]) * sorted_u_i)
            if Epos > Eneg:
                if Ein_i > Eneg:
                    Ein_i = Eneg
                    s_i = -1
                    threshold_i = threshold
            else:
                if Ein_i > Epos:
                    Ein_i = Epos
                    s_i = 1
                    threshold_i = threshold

        if Ein > Ein_i:
            Ein = Ein_i
            s = s_i
            thres = threshold_i
            feature_idx = i

    u, alpha = update_u(u,s,thres,feature_idx, y_train)
    classifiers.append([alpha, s, thres, feature_idx])
    
    
    Ein_G = Ein_Gt(classifiers)
    if Ein_G <= 0.05:
        print(t)
        break

60 0.117
160 0.071
260 0.06
360 0.052


In [17]:
#Q14
N = x_train.shape[0]
dim = x_train.shape[1]
u = np.array([(1.0 / N)] * N)
Ein = 1
s = 1
thres = 0
feature_idx = 0

for i in range(dim):
    Ein_i = 1
    s_i = 1
    threshold_i = 0
    feature_i = x_train[:,i]
    data_i = np.transpose(np.array([feature_i, y_train]))
    sorted_data_i = data_i[np.argsort(data_i[:, 0])]
    sorted_u_i = u[np.argsort(data_i[:, 0])]
    
    thresholds = np.array([float("-inf")] + [(sorted_data_i[j,0] + sorted_data_i[j+1,0]) / 2 for j in range(N-1)])
    
    for threshold in thresholds:
        y_pos = np.where(sorted_data_i[:,0] > threshold, 1, -1)
        y_neg = np.where(sorted_data_i[:,0] < threshold, 1, -1)
        Epos = sum((y_pos != sorted_data_i[:,1]) * u)
        Eneg = sum((y_neg != sorted_data_i[:,1]) * u)
        if Epos > Eneg:
            if Ein_i > Eneg:
                Ein_i = Eneg
                s_i = -1
                threshold_i = threshold
        else:
            if Ein_i > Epos:
                Ein_i = Epos
                s_i = 1
                threshold_i = threshold
    
    if Ein > Ein_i:
        Ein = Ein_i
        s = s_i
        thres = threshold_i
        feature_idx = i

feature_i = x_test[:,feature_idx]
data_i = np.transpose(np.array([feature_i, y_test]))

if s == -1:
    y_pred = np.where(data_i[:,0] < thres, 1, -1)
else:
    y_pred = np.where(data_i[:,0] > thres, 1, -1)
Eout = sum(y_pred != data_i[:,1]) / x_test.shape[0]

print(Eout)

#choose b

0.455


In [3]:
#Q15
def Eout_G_uniform(classifiers):
    x_test_label = np.array([0.0] * x_test.shape[0])
    for classifier in classifiers:
        if classifier[1] == 1:
            pred_y = np.where(x_test[:,classifier[3]] > classifier[2], 1, -1)
        else:
            pred_y = np.where(x_test[:,classifier[3]] < classifier[2], 1, -1)
        
        x_test_label += pred_y
    x_test_label = np.where(x_test_label > 0, 1, -1)
    Eout_G = sum(x_test_label != y_test)/ x_test.shape[0]
    return Eout_G

#Q16
def Eout_G(classifiers):
    x_test_label = np.array([0.0] * x_test.shape[0])
    for classifier in classifiers:
        if classifier[1] == 1:
            pred_y = np.where(x_test[:,classifier[3]] > classifier[2], 1, -1)
        else:
            pred_y = np.where(x_test[:,classifier[3]] < classifier[2], 1, -1)
        
        x_test_label += classifier[0]*pred_y
    x_test_label = np.where(x_test_label > 0, 1, -1)
    Eout_G = sum(x_test_label != y_test)/ x_test.shape[0]
    return Eout_G

In [4]:
#Q15 16
classifiers = []
N = x_train.shape[0]
dim = x_train.shape[1]
u = np.array([(1.0 / N)] * N)
s = 1
thres = 0
max_Ein = 0
feature_idx = 0

for t in range(500):
    Ein = 1
    for i in range(dim):
        Ein_i = 1
        threshold_i = 0
        data_i = np.transpose(np.array([x_train[:,i], y_train]))
        sorted_data_i = data_i[np.argsort(data_i[:, 0])]
        sorted_u_i = u[np.argsort(data_i[:, 0])]

        thresholds = np.array([float("-inf")] + [(sorted_data_i[j,0] + sorted_data_i[j+1,0]) / 2 for j in range(N-1)])

        for threshold in thresholds:
            y_pos = np.where(sorted_data_i[:,0] > threshold, 1, -1)
            y_neg = np.where(sorted_data_i[:,0] < threshold, 1, -1)
            Epos = sum((y_pos != sorted_data_i[:,1]) * sorted_u_i)
            Eneg = sum((y_neg != sorted_data_i[:,1]) * sorted_u_i)
            if Epos > Eneg:
                if Ein_i > Eneg:
                    Ein_i = Eneg
                    s_i = -1
                    threshold_i = threshold
            else:
                if Ein_i > Epos:
                    Ein_i = Epos
                    s_i = 1
                    threshold_i = threshold

        if Ein > Ein_i:
            Ein = Ein_i
            s = s_i
            thres = threshold_i
            feature_idx = i

    u, alpha = update_u(u,s,thres,feature_idx, y_train)
    classifiers.append([alpha, s, thres, feature_idx])
    print('classifier ',t, ': ',  'alpha: ', alpha, 'sign: ', s, 'threshold: ', thres, 'fearure_idx: ',feature_idx)

Eout_G_u = Eout_G_uniform(classifiers)
Eout_G = Eout_G(classifiers)
print('Q15: ', Eout_G_u)
print('Q16: ', Eout_G)
    
#choose a
#choose b

classifier  0 :  alpha:  0.25754728684278305 sign:  -1 threshold:  0.44824087255 fearure_idx:  9
classifier  1 :  alpha:  0.21176853641688023 sign:  1 threshold:  2.6792002063 fearure_idx:  0
classifier  2 :  alpha:  0.20516252795484194 sign:  -1 threshold:  -1.51380683245 fearure_idx:  0
classifier  3 :  alpha:  0.28283938717375584 sign:  1 threshold:  -1.24708306405 fearure_idx:  5
classifier  4 :  alpha:  0.17673533777802233 sign:  1 threshold:  3.8257959154 fearure_idx:  0
classifier  5 :  alpha:  0.18752999899129483 sign:  1 threshold:  -0.5865877720499999 fearure_idx:  8
classifier  6 :  alpha:  0.1790955306893439 sign:  -1 threshold:  -2.25328903845 fearure_idx:  0
classifier  7 :  alpha:  0.20138746519881062 sign:  1 threshold:  -0.67601466975 fearure_idx:  7
classifier  8 :  alpha:  0.1303202175848111 sign:  1 threshold:  2.94708224915 fearure_idx:  5
classifier  9 :  alpha:  0.18006092085537057 sign:  -1 threshold:  0.5180065872999999 fearure_idx:  8
classifier  10 :  alpha: 

KeyboardInterrupt: 